### Task 1 (20% credit)

Assume you have a data set as below. It contains records of cars with three features: the type of the car (sports(1) or SUV(2)), the color of the car (red(1) or yellow(2)), and the origin of the car (domestic(1) or imported(2)). And the labels for the data are: stolen(1) and not(0). 

#### Questions:

a) Calculate the following sample probabilities:
P(Red|Stolen), P(SUV|Stolen), P(Domestic|Stolen), P(Red|Not Stolen) , P(SUV|Not Stolen), and P(Domestic|Not Stolen)

b) Suggest a classification for a red, domestic SUV - whether it will be stolen or not - using Naive Bayes classifier. 

Please perform all the necessary computations "by hands" rather than using python code.

In [1]:
import numpy as np
import pandas as pd
from scipy import stats
import math
y=[1,0,1,0,1,0,1,0,0,1]
X=[[1,1,1,2,2,2,2,2,1,1],[1,1,1,1,1,2,2,2,2,1],[1,1,1,1,2,2,2,1,2,2]]
data=[y]+X
data=pd.DataFrame(data).T
data.columns=['Stolen?','Color','Type','Origin']

__(a)__

In [2]:
print "P(Stolen) = ", 1.*len(data[(data['Stolen?']==1)])/len(data)
print "P(Red|Stolen) = ", 1.*len(data[(data['Stolen?']==1)&(data['Color']==1)])/len(data[data['Stolen?']==1])
print "P(SUV|Stolen) = ", 1.*len(data[(data['Stolen?']==1)&(data['Type']==2)])/len(data[data['Stolen?']==1])
print "P(Domestic|Stolen) = ", 1.*len(data[(data['Stolen?']==1)&(data['Origin']==1)])/len(data[data['Stolen?']==1])
print "P(Red|Not Stolen) = ", 1.*len(data[(data['Stolen?']==0)&(data['Color']==1)])/len(data[data['Stolen?']==0])
print "P(SUV|Not Stolen) = ", 1.*len(data[(data['Stolen?']==0)&(data['Type']==2)])/len(data[data['Stolen?']==0])
print "P(Domestic|Not Stolen) = ", 1.*len(data[(data['Stolen?']==0)&(data['Origin']==1)])/len(data[data['Stolen?']==0])

P(Stolen) =  0.5
P(Red|Stolen) =  0.6
P(SUV|Stolen) =  0.2
P(Domestic|Stolen) =  0.4
P(Red|Not Stolen) =  0.4
P(SUV|Not Stolen) =  0.6
P(Domestic|Not Stolen) =  0.6


__(b)__

In [3]:
print "P(Stolen|{Red, Domestic, SUV}) = P(Red|Stolen) * P(Domestic|Stolen) * P(SUV|Stolen) * P(Stolen) = ", .6*.4*.2*.5
print "P(Not Stolen|{Red, Domestic, SUV}) = P(Red|Not Stolen) * P(Domestic|Not Stolen) * P(SUV|Not Stolen) * P(Not Stolen) = ", .4*.6*.6*.5

P(Stolen|{Red, Domestic, SUV}) = P(Red|Stolen) * P(Domestic|Stolen) * P(SUV|Stolen) * P(Stolen) =  0.024
P(Not Stolen|{Red, Domestic, SUV}) = P(Red|Not Stolen) * P(Domestic|Not Stolen) * P(SUV|Not Stolen) * P(Not Stolen) =  0.072


P(Not Stolen|{Red, Domestic, SUV}) > P(Stolen|{Red, Domestic, SUV}) -> Not Stolen

### Task 2 (25% credit)
Consider a following Guassian Naive Bayes problem.
We use eight factors to predict if people have diabetes or not. The variabls are:

y: The label (0 - no diabetes, 1 - diabetes)

t_pre: Number of times pregnant

glu: Plasma glucose concentration a 2 hours in an oral glucose tolerance test

blood_p: Diastolic blood pressure (mm Hg)

triceps: Triceps skin fold thickness (mm)

serum: 2-Hour serum insulin (mu U/ml)

b_m: Body mass index (weight in kg/(height in m)^2)

pedigree_f: Diabetes pedigree function

age: Age (years)
#### ---------------------------------------------------------------------------------------------------------------------

#### Questions:

a)Train the classifier: use the training data to estimate prior probabilities $P(y=b)$ as well as the parameters (mean and standard deviation) of the sample distributions $P(x_i|y=b)$.

b)Perform the classification for the test sample. 

c)Compare your result to y_test and report the classification accuracy.

__(a)__

In [4]:
data_train=pd.read_csv("dia_train.csv", usecols=range(1,10)) 

data_test=pd.read_csv("dia_test.csv", usecols=range(1,10))

In [5]:
def trainNaiveBayes(trainData):
  #training Gausian Naive Bayes Classifier
  tY=trainData.loc[:,trainData.columns[0]]
  ind1=tY==0
  ind2=tY==1
  dp=pd.DataFrame(columns=trainData.columns, index=['mu1','sigma1','mu2','sigma2'])
  #estimate priors
  dp[trainData.columns[0]]['mu1']=1.0*sum(ind1)/len(trainData.index)
  dp[trainData.columns[0]]['mu2']=1.0*sum(ind2)/len(trainData.index)
  #estimate sample distribution paramters for p(xi|y=b)
  for i in trainData.columns[1:]:
    dp.loc['mu1',i]=(trainData[i][ind1]).mean()
    dp.loc['sigma1',i]=(trainData[i][ind1]).std()
    dp.loc['mu2',i]=(trainData[i][ind2]).mean()
    dp.loc['sigma2',i]=(trainData[i][ind2]).std()
  return dp

def classifyNaiveBayes(classData,dp):
  #classifying using trained Gausian Naive Bayes Classifier
  Y=classData.loc[:,classData.columns[0]]*0
  for j in classData.index:
    #start from the priors
    P1=dp[classData.columns[0]]['mu1'];
    P2=dp[classData.columns[0]]['mu2'];
    #multiply by conditional probability densities p(xi|y=b)
    for i in classData.columns[1:]:
        if dp[i]['sigma1']==0: #if sigma can not be defined (sample does not have variance)
            P1=P1*stats.norm.pdf(classData[i][j], loc=dp[i]['mu1'],scale=1) #pick up arbitrary sigma if undefined
        else:
            P1=P1*stats.norm.pdf(classData[i][j], loc=dp[i]['mu1'],scale=dp[i]['sigma1'])
        
        if dp[i]['sigma2']==0: #if sigma can not be defined (sample does not have variance)
            P2=P2*stats.norm.pdf(classData[i][j], loc=dp[i]['mu2'],scale=1) #pick up arbitrary sigma if undefined
        else:
            P2=P2*stats.norm.pdf(classData[i][j], loc=dp[i]['mu2'],scale=dp[i]['sigma2']) 
    Y[j]=int(P2>P1)
 

  return Y

In [6]:
dp=trainNaiveBayes(data_train)

In [7]:
dp

,y,t_pre,glu,blood_p,triceps,serum,b_m,pedigree_f,age
mu1,0.6991525,2.690909,111.4667,69.20606,27.2,127.0061,31.70909,0.4686848,28.39394
sigma1,NaN,2.610061,24.69189,11.71329,10.43692,91.48614,6.337613,0.2917503,8.537362
mu2,0.3008475,4.070423,144.1408,74.56338,33.47887,209.2113,35.22394,0.6390423,35.78873
sigma2,NaN,3.51862,30.62646,13.79931,9.762697,126.921,6.258491,0.439042,10.26355


__(b)__

In [8]:
C=classifyNaiveBayes(data_test,dp)

__(c)__

In [9]:
print "Classification accuracy for test set =", 1.0*sum(C==data_test.y)/len(C)

Classification accuracy for test set = 0.791139240506


### Task 3 (Credit 25%)
We have an artificial data set split, while the training set contains both - labeled (Label_train) and unlabeled (Unlabel) data. Column 'y' is the label, and columns '0','1','2' are categorical variables.

#### Questions:

a) Use the labeled part data_train to predict the labels of X_Label_test, and report the classification accuracy.

b) Improve the classification by using the unlabeled data data_Unlabel and the EM algorithm to predict labels of X_Label_test, and report the new accuracy by EM semi-supervised algorithm (use the same convergence criteria as in the lecture notebook). 

In [10]:
data_train=pd.read_csv("EM_train.csv", usecols=range(1,5))
data_test=pd.read_csv("EM_test.csv", usecols=range(1,5))
data_Unlabel=pd.read_csv("EM_Unlabel.csv", usecols=range(1,4))

In [11]:
def trainNaiveBayesDiscrete(trainData):
  #training discrete Naive Bayes Classifier
  tY=trainData.loc[:,trainData.columns[0]]
  m=max([trainData[j][i] for j in trainData.columns[1:] for i in trainData.index]) #maximal number of classes in each feature of a training set
  #create output data structure for the probabilities - same column labels, rows correspond to values of x and there are two arrays like that for different b
  dp=[pd.DataFrame(columns=trainData.columns, index=range(1,m+1)), pd.DataFrame(columns=trainData.columns, index=range(1,m+1))]
  #split the training data between two labels
  ind1=tY==0
  ind2=tY==1
  #estimate P(y=b)  
  dp[0][trainData.columns[0]][1]=1.0*ind1.sum()/len(trainData.index)
  dp[1][trainData.columns[0]][1]=1.0*ind2.sum()/len(trainData.index)
  #estimate conditional probabilities P(x|y=b)
  for j in trainData.columns[1:]:
    for i in range(1,m+1):
        dp[0].loc[i,j]=1.0*(trainData[j][ind1]==i).sum()/ind1.sum();
        dp[1].loc[i,j]=1.0*(trainData[j][ind2]==i).sum()/ind2.sum();
  return dp

def classifyNaiveBayesDiscrete(classData,dp):
  #classifying using trained discrete Naive Bayes Classifier
  Y=classData[classData.columns[0]]*0 #initialize the empty array 
  for i in classData.index: #for al records to classify
    #start with the priors
    P1=dp[0][classData.columns[0]][1]; 
    P2=dp[1][classData.columns[0]][1];
    #and multiply them by the corresponding conditional probabilities P(x_i|y=b)
    for j in classData.columns[1:]:
      P1=P1*dp[0][j][classData[j][i]]
      P2=P2*dp[1][j][classData[j][i]]
    Y[i]=int(P2>P1) #finally for each record decide which P(y|x) is higher and choose the label
  return Y

__(a)__

In [12]:
dp=trainNaiveBayesDiscrete(data_train)
C=classifyNaiveBayesDiscrete(data_test,dp)
print "Classification accuracy for test set =", 1.0*sum(C==data_test.y)/len(C)

Classification accuracy for test set = 0.638888888889


__(b)__

In [13]:
#implementation of Expectation-Maximization algorithm for partially labeled data
def EM(X_Label,y_Label,X_Unlabel,dp):
  t = 0  
  haslabels=len(y_Label)>0

  while True:
    t = t + 1

    classData=X_Unlabel
    # Now we want to calculate P(y=1|x) and P(y=2|x) for all observations xj. (these are bunch of scalars)
    # we need this to calculate new dp. Basically speaking, for every new iteration we need a new dp.

    #for y=1 and y=2

    p_x_1=[] #unnormalized P(y=1|x)
    p_x_2=[] #unnormalized P(y=2|x)
    cols=dp[0].columns

    for i in classData.index:
        P1=dp[0][cols[0]][1];
        P2=dp[1][cols[0]][1];
        for j in classData.columns:
            P1=P1*dp[0][j][classData[j][i]]
            P2=P2*dp[1][j][classData[j][i]]
        p_x_1.append(P1)
        p_x_2.append(P2)

    #Rescale p_x_1 and p_x_2:
    summ=np.asarray(p_x_1)+np.asarray(p_x_2)
    p_x_1_s=np.asarray(p_x_1)/summ
    p_x_2_s=np.asarray(p_x_2)/summ
    inds_1 = np.where(np.isnan(p_x_1_s))
    inds_2 = np.where(np.isnan(p_x_2_s))
    p_x_1_s[inds_1]=0.5
    p_x_2_s[inds_2]=0.5
    #Now let's calculate P(y=1) and P(y=2)
    p_1=p_x_1_s.sum()/len(p_x_1_s)
    p_2=p_x_2_s.sum()/len(p_x_2_s)


    #Now let's calculate the probability distribution of P(xi|y=1) and P(xi|y=2)
    
    m=max([classData[j][i] for j in classData.columns for i in classData.index]) #maximal number of classes in each feature of a training set

    #create output data structure for the probabilities - new iteration
    
    dp1=[pd.DataFrame(columns=cols, index=range(1,m+1)), pd.DataFrame(columns=cols, index=range(1,m+1))]

    #P(y=b)  
    dp1[0][cols[0]][1]=p_1
    dp1[1][cols[0]][1]=p_2


    #estimate conditional probabilities P(x|y=b) -do we add labeled data to fit?

    temp=np.concatenate((np.asmatrix(X_Unlabel),np.asarray(pd.DataFrame(p_x_1_s)),np.asarray(pd.DataFrame(p_x_2_s))), axis=1)
    temp=pd.DataFrame(temp)
    if haslabels:
        temp_l=np.concatenate((np.asmatrix(X_Label),np.asmatrix(1*(y_Label==0)).transpose(),np.asmatrix(1*(y_Label==1)).transpose()),axis=1)
        temp_l=pd.DataFrame(temp_l)
        pd.concat([temp,temp_l])
   

    for j in range(1,len(dp[0].T)):
        for i in range(len(dp[0])):

            dp1[0].iloc[i,j]=temp[temp.iloc[:,j-1]==i+1].iloc[:,-2].sum()/temp.iloc[:,-2].sum()
            dp1[1].iloc[i,j]=temp[temp.iloc[:,j-1]==i+1].iloc[:,-1].sum()/temp.iloc[:,-1].sum()

        ############################################################################################
    # Now we use dp to decide whether to continue our iterations
    #print dp[0]
    #print np.sum((dp1[0] - dp[0])**2)
    #print dp[1]
    #print np.sum((dp1[1] - dp[1])**2)
    if (sum(np.sum((dp1[0]-dp[0])**2))+sum(np.sum((dp1[1]-dp[1])**2)))<0.001: #if dp does not change much
        break
    else: 
        dp=dp1  #save new dp and perform next iteration

        
    ###############################################################################################
        #Calculate the log-likelihood
        
        L=0
        
        for i in classData.index:
            P1=dp[0][cols[0]][1];
            P2=dp[1][cols[0]][1];
            for j in classData.columns:
                P1=P1*dp[0][j][classData[j][i]]
                P2=P2*dp[1][j][classData[j][i]]
            temp=math.log(P1+P2)
            L=L+temp
        if haslabels:    
          for i in X_Label.index:
            yi=y_Label[i]
            P=dp[yi][cols[0]][1];
            for j in X_Label.columns:
                P=P*dp[yi][j][X_Label[j][i]]
            L=L+math.log(P)
        
        #print "Iteration {0}: log maximum liklihood = {1}".format(t,L)    
        
        
  return dp

In [14]:
dpEM = EM(data_train.iloc[:,1:], data_train.iloc[:,0], data_Unlabel, dp)
C=classifyNaiveBayesDiscrete(data_test,dpEM) #classify test data with a new theta given by EM
print "Classification accuracy for test set after EM =", 1.0*sum(C==data_test.y)/len(C)

Classification accuracy for test set after EM = 0.944444444444


### Task 4 (Credit 30%)
For the similar artifitial data uploaded below:

#### Question: 

a) Apply the EM algorithm (no observed labels, random initial choice of $\theta$) for clustering the data records into two clusters. Report your result (a vector of cluster numbers for each data record). 

b) Repeat the clustering 10 times with different random choices of $\theta$ and analyze the stability of the clustering (matching labels accross different clusterings (use the choice of 0 and 1 labels best matching the previous clustering), estimate average label and its standard error for each record).

__(a)__

In [15]:
np.random.seed(42)

data=pd.read_csv("EM_Cluster.csv")
X=data.iloc[:,1:]

# Random Ys
Y = np.random.randint(0,2,len(X))
X.insert(0, 'y', Y)

dp=trainNaiveBayesDiscrete(X)
dpEM = EM([], [], X.iloc[:,1:], dp)
C=classifyNaiveBayesDiscrete(X,dpEM)

print C

0      0
1      0
2      0
3      1
4      0
5      0
6      1
7      0
8      0
9      1
10     1
11     1
12     1
13     0
14     0
15     0
16     1
17     0
18     1
19     1
20     1
21     0
22     1
23     1
24     0
25     1
26     0
27     0
28     0
29     1
      ..
75     0
76     0
77     1
78     0
79     1
80     1
81     1
82     0
83     1
84     1
85     0
86     0
87     0
88     0
89     1
90     1
91     0
92     1
93     1
94     0
95     1
96     0
97     1
98     1
99     0
100    1
101    1
102    1
103    0
104    0
Name: y, dtype: int32


__(b)__

In [16]:
results = pd.DataFrame()

for i in range(10):
    data=pd.read_csv("EM_Cluster.csv")
    X=data.iloc[:,1:]

    # Random Ys
    Y = np.random.randint(0,2,len(X))
    X.insert(0, 'y', Y)

    dp=trainNaiveBayesDiscrete(X)
    dpEM = EM([], [], X.iloc[:,1:], dp)
    C=classifyNaiveBayesDiscrete(X,dpEM)
    results[i] = C

In [17]:
# Correct for opposite labelings by assuming a stability of > 75
while np.any(np.sum(np.abs(np.diff(results)), axis=0) > 75):
    # Get the first column which is opposite of the previous
    i = np.argmax(np.sum(np.abs(np.diff(results)), axis=0) > 75)
    
    # Invert it
    results[i] = (~results[i].astype(bool)).astype(int)

In [18]:
# Print the means and errors of the elements which are not stable
means = results.mean(axis=1)
print "means:"
print means[(means>0)&(means<1)]

se = results.std(axis=1)/math.sqrt(10)
print "\nstandard errors:"
print se[(means>0)&(means<1)]

means:
16    0.8
22    0.1
32    0.1
47    0.7
56    0.7
75    0.7
89    0.1
93    0.1
dtype: float64

standard errors:
16    0.133333
22    0.100000
32    0.100000
47    0.152753
56    0.152753
75    0.152753
89    0.100000
93    0.100000
dtype: float64
